In [1]:
from carla_visual.dataloaders import load_online_dataset, pytorch_dataset_to_tf_dataset

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [2]:
ds_name = 'MNIST'
data_root = 'D:\PycharmProjects\XAIRobustness\data\images'

input_shape = (28, 28, 1)
batch_size = 64
num_classes = 10

train_data, test_data = load_online_dataset(ds_name, data_root, download=False, split_val=False)
print(f'Shape of the train set: ', train_data.data.shape)
print(f'Shape of the test set: ', test_data.data.shape)

Shape of the train set:  torch.Size([60000, 28, 28])
Shape of the test set:  torch.Size([10000, 28, 28])


In [3]:
train_dataset = pytorch_dataset_to_tf_dataset(train_data, input_shape, num_classes=num_classes)
test_dataset = pytorch_dataset_to_tf_dataset(test_data, input_shape,  num_classes=num_classes)

train_dataset = train_dataset.shuffle(10000).batch(batch_size)
test_dataset = test_dataset.shuffle(10000).batch(batch_size)


# Load weights of a classifier and an autoencoder

If the weights are absent, use the helper functions for training 

In [24]:
from carla_visual.recourse_methods.countergan import model
import carla_visual.recourse_methods.countergan.train_functions as ctf

In [25]:
from tensorflow.keras import layers, models, optimizers


class SimpleCNN(models.Sequential):
    def __init__(self, num_classes=10, **kwargs):
        super(SimpleCNN, self).__init__(**kwargs)

        self.add(layers.Conv2D(64, kernel_size=2, padding='same', activation='relu',
                               input_shape=(28,28,1),
                               name='conv2d_1'))
        self.add(layers.MaxPooling2D(pool_size=2, name='maxpool_1'))
        self.add(layers.Dropout(0.3, name='drpt_1'))

        self.add(layers.Conv2D(32, kernel_size=2, padding='same', activation='relu', name='conv2d_2'))
        self.add(layers.MaxPooling2D(pool_size=2, name='maxpool_2'))
        self.add(layers.Dropout(0.3, name='drpt_2'))

        self.add(layers.Flatten(name='flatten_1'))
        self.add(layers.Dense(256, activation='relu', name='dense_1'))
        self.add(layers.Dropout(0.5, name='drpt_3'))
        self.add(layers.Dense(num_classes, activation='softmax', name='dense_2'))

        self.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

In [29]:
cnn = SimpleCNN(num_classes)

In [30]:
cnn.summary()

Model: "simple_cnn_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 64)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_1 (MaxPooling2D)        │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drpt_1 (Dropout)                │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 14, 14, 32)     │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_2 (MaxPooling2D)        │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drpt_2 (Dropout)                │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1568)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       401,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drpt_3 (Dropout)                │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 412,778 (1.57 MB)

 Trainable params: 412,778 (1.57 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
cnn_1 = model.SimpleCNN(num_classes)
cnn_1.summary()

Model: "simple_cnn_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool1 (MaxPooling2D)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drpt1 (Dropout)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool2 (MaxPooling2D)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drpt2 (Dropout)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten1 (Flatten)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drpt3 (Dropout)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:
cnn_weights_path = r'D:\PycharmProjects\XAIRobustness\CARLA_VISUAL\carla_visual\models\weights\countergan'
cnn_name = 'cnn'

# the argument validation_data of the fit() does not accept the Dataset type objects
cnn = ctf.train_classifier(cnn, train_dataset=train_dataset, 
                           val_dataset=(test_data.data.numpy(),
                                        to_categorical(test_data.targets.numpy())), 
                           epochs=2, name=cnn_name,
                           weights_path_to_save=cnn_weights_path)

Epoch 1/2
    937/Unknown 36s 33ms/step - accuracy: 0.8125 - loss: 0.5703

C:\Users\sahat\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


938/938 ━━━━━━━━━━━━━━━━━━━━ 37s 34ms/step - accuracy: 0.8127 - loss: 0.5697 - val_accuracy: 0.9454 - val_loss: 8.8863
Epoch 2/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.9614 - loss: 0.1250 - val_accuracy: 0.9618 - val_loss: 5.5699


Training: loss=0.1129, accuracy=0.9649
Validation: loss=5.5699, accuracy=0.9618


In [33]:
cnn_weights_path = r'D:\PycharmProjects\XAIRobustness\CARLA_VISUAL\carla_visual\models\weights\countergan'
cnn_name = 'cnn'
model = tf.keras.models.load_model(f'{cnn_weights_path}\{cnn_name}.h5', custom_objects={'SimpleCNN': SimpleCNN})

d:\PycharmProjects\XAIRobustness\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Sequential model 'simple_cnn_1' has already been configured to use input shape (None, 28, 28, 1). You cannot add a different Input layer to it.

In [29]:
autoencoder = model.Autoencoder(input_shape)

In [30]:
ae_weights_path = r'D:\PycharmProjects\XAIRobustness\CARLA_VISUAL\carla_visual\models\weights\countergan'

ctf.train_autoencoder(autoencoder, 
                      train_data.data.numpy(),
                      test_data.data.numpy(), 
                      ae_weights_path)

ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: 28 (of type <class 'int'>)